In [16]:
pip install scikit-fuzzy


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [17]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

Generem dades artificials que de moment no tenim les oficials

In [18]:
import pandas as pd
import numpy as np

# Llista dels 51 estats dels EUA (noms complets)
estats_usa = [
    "Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", "Delaware",
    "District of Columbia", "Florida", "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa",
    "Kansas", "Kentucky", "Louisiana", "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota",
    "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire", "New Jersey", "New Mexico",
    "New York", "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island",
    "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington",
    "West Virginia", "Wisconsin", "Wyoming"
]
# Genera dades aleatòries realistes (ajustables)
np.random.seed(42)  # Per reprodibilitat

# Genera valors aleatoris per hospitalitzats, morts i contagiats
df_epidemiologic = pd.DataFrame({
    "state": estats_usa,
    "hospitalitzats": np.random.randint(50, 2000, size=51),  # Exemple: entre 50 i 2000
    "morts": np.random.randint(5, 500, size=51),            # Exemple: entre 5 i 500
    "contagiats": np.random.randint(1000, 50000, size=51)   # Exemple: entre 1000 i 50000
})

# Mostra les primeres files
print(df_epidemiologic.head())

        state  hospitalitzats  morts  contagiats
0     Alabama            1176     55       34763
1      Alaska            1509    368       33606
2     Arizona             910     59       12534
3    Arkansas            1344    248       30127
4  California            1180    324       41397


In [19]:
import pandas as pd

df_demografic_original = pd.read_csv("../data/preprocessed/dataMatrix/static_stateMatrix.csv")
columnes = [
    "state", 
    "population_state",
    "pop_density_state",
    "bedsTotal",
    "pop_0-9", "pop_10-19", "pop_20-29", "pop_30-39",
    "pop_40-49", "pop_50-59", "pop_60-69", "pop_70-79", "pop_80+"
]
df_demografic = df_demografic_original[columnes]

Fusionem per tenir dades població (x edad ens poden servir també x tenir zones amb molta gent gran)

In [20]:
# Fusiona els dos DataFrames per la columna "state"
df_combinat = pd.merge(
    df_demografic, 
    df_epidemiologic, 
    on="state", 
    how="inner" 
)

# Mostra les primeres files (opcional)
print(df_combinat.head())

        state  population_state  pop_density_state  bedsTotal   pop_0-9  \
0     Alabama           4887871          96.509389        3.1  0.120933   
1      Alaska            737438           1.291523        2.2  0.142617   
2     Arizona           7171646          63.135855        1.9  0.123774   
3    Arkansas           3013825          57.919684        3.2  0.127289   
4  California          39557045         253.906502        1.8  0.124635   

   pop_10-19  pop_20-29  pop_30-39  pop_40-49  pop_50-59  pop_60-69  \
0   0.128108   0.136099   0.124291   0.123013   0.133249   0.120537   
1   0.130210   0.155501   0.147626   0.117347   0.127204   0.109072   
2   0.132496   0.140854   0.129224   0.119039   0.120016   0.113492   
3   0.132222   0.135421   0.126671   0.119787   0.126968   0.115755   
4   0.128760   0.147575   0.143412   0.128138   0.126974   0.104040   

   pop_70-79   pop_80+  hospitalitzats  morts  contagiats  
0   0.075243  0.038527            1176     55       34763  
1 

In [21]:
df_combinat["bedsTotal"]

0     3.1
1     2.2
2     1.9
3     3.2
4     1.8
5     1.9
6     2.0
7     2.2
8     4.4
9     2.6
10    2.4
11    1.9
12    1.9
13    2.5
14    2.7
15    3.0
16    3.3
17    3.2
18    3.3
19    2.5
20    1.9
21    2.3
22    2.5
23    2.5
24    4.0
25    3.1
26    3.3
27    3.6
28    2.1
29    2.1
30    2.4
31    1.8
32    2.7
33    2.1
34    4.3
35    2.8
36    2.8
37    1.6
38    2.9
39    2.1
40    2.4
41    4.8
42    2.9
43    2.3
44    1.8
45    2.1
46    2.1
47    1.7
48    3.8
49    2.1
50    3.5
Name: bedsTotal, dtype: float64

In [22]:
# IA (Incidència Acumulada)
df_combinat["IA"] = (df_combinat["contagiats"] / df_combinat["population_state"]) * 100000

# θ (Ocupació hospitalària %)
df_combinat["beds_real"] = (df_combinat["bedsTotal"] * df_combinat["population_state"]) / 1000
df_combinat["theta"] = (df_combinat["hospitalitzats"] / df_combinat["beds_real"]) * 100

# π (Taxa de mortalitat %) per 100000 habitants
df_combinat["pi"] = (df_combinat["morts"] / df_combinat["population_state"]) * 100000

# % població >65
df_combinat["pop_>65"] = df_combinat["pop_70-79"] + df_combinat["pop_80+"]


In [23]:
df_combinat.head()


,state,population_state,pop_density_state,bedsTotal,pop_0-9,pop_10-19,pop_20-29,pop_30-39,pop_40-49,pop_50-59,...,pop_70-79,pop_80+,hospitalitzats,morts,contagiats,IA,beds_real,theta,pi,pop_>65
0,Alabama,4887871,96.509389,3.1,0.120933,0.128108,0.136099,0.124291,0.123013,0.133249,...,0.075243,0.038527,1176,55,34763,711.209441,15152.4001,7.761147,1.125234,0.113770
1,Alaska,737438,1.291523,2.2,0.142617,0.130210,0.155501,0.147626,0.117347,0.127204,...,0.050675,0.019748,1509,368,33606,4557.128870,1622.3636,93.012442,49.902500,0.070424
2,Arizona,7171646,63.135855,1.9,0.123774,0.132496,0.140854,0.129224,0.119039,0.120016,...,0.080306,0.040799,910,59,12534,174.771594,13626.1274,6.678346,0.822684,0.121105
3,Arkansas,3013825,57.919684,3.2,0.127289,0.132222,0.135421,0.126671,0.119787,0.126968,...,0.075878,0.040010,1344,248,30127,999.626720,9644.2400,13.935779,8.228746,0.115888
4,California,39557045,253.906502,1.8,0.124635,0.128760,0.147575,0.143412,0.128138,0.126974,...,0.060900,0.035566,1180,324,41397,104.651396,71202.6810,1.657241,0.819070,0.096466


In [26]:
df_demografic_original.columns

Index(['state', 'no_coverage', 'private_coverage', 'public_coverage',
       'labor_cov_diff', 'bedsState_local_government', 'bedsNon_profit',
       'bedsFor_profit', 'bedsTotal', 'population_state', 'pop_density_state',
       'pop_0-9', 'pop_10-19', 'pop_20-29', 'pop_30-39', 'pop_40-49',
       'pop_50-59', 'pop_60-69', 'pop_70-79', 'pop_80+', 'Low_SVI_CTGY',
       'Moderate_Low_SVI_CTGY', 'Moderate_High_SVI_CTGY', 'High_SVI_CTGY',
       'Metro', 'Non-metro'],
      dtype='object')

In [27]:
df_demografic.columns

Index(['state', 'population_state', 'pop_density_state', 'bedsTotal',
       'pop_0-9', 'pop_10-19', 'pop_20-29', 'pop_30-39', 'pop_40-49',
       'pop_50-59', 'pop_60-69', 'pop_70-79', 'pop_80+'],
      dtype='object')

In [28]:
df_combinat.columns

Index(['state', 'population_state', 'pop_density_state', 'bedsTotal',
       'pop_0-9', 'pop_10-19', 'pop_20-29', 'pop_30-39', 'pop_40-49',
       'pop_50-59', 'pop_60-69', 'pop_70-79', 'pop_80+', 'hospitalitzats',
       'morts', 'contagiats', 'IA', 'beds_real', 'theta', 'pi', 'pop_>65',
       'risc'],
      dtype='object')

In [29]:
df_combinat.head()

,state,population_state,pop_density_state,bedsTotal,pop_0-9,pop_10-19,pop_20-29,pop_30-39,pop_40-49,pop_50-59,...,pop_80+,hospitalitzats,morts,contagiats,IA,beds_real,theta,pi,pop_>65,risc
0,Alabama,4887871,96.509389,3.1,0.120933,0.128108,0.136099,0.124291,0.123013,0.133249,...,0.038527,1176,55,34763,711.209441,15152.4001,7.761147,1.125234,0.113770,4.352421
1,Alaska,737438,1.291523,2.2,0.142617,0.130210,0.155501,0.147626,0.117347,0.127204,...,0.019748,1509,368,33606,4557.128870,1622.3636,93.012442,49.902500,0.070424,5.533006
2,Arizona,7171646,63.135855,1.9,0.123774,0.132496,0.140854,0.129224,0.119039,0.120016,...,0.040799,910,59,12534,174.771594,13626.1274,6.678346,0.822684,0.121105,3.479155
3,Arkansas,3013825,57.919684,3.2,0.127289,0.132222,0.135421,0.126671,0.119787,0.126968,...,0.040010,1344,248,30127,999.626720,9644.2400,13.935779,8.228746,0.115888,4.995570
4,California,39557045,253.906502,1.8,0.124635,0.128760,0.147575,0.143412,0.128138,0.126974,...,0.035566,1180,324,41397,104.651396,71202.6810,1.657241,0.819070,0.096466,5.223613


## Aplicació de regles de coneixement difús 

In [ ]:

def classifica_risc(row):
    if row["Rt"] > 1.5 and row["IA"] > 150 and row["theta"] > 80:
        return "Crític"
    elif row["Rt"] > 1.2 and (row["IA"] > 100 or row["pi"] > 1.5) and row["theta"] > 70:
        return "Alt"
    elif row["IA"] > 50 and row["positivity_rate"] > 5:
        return "Moderat"
    else:
        return "Baix"

df_combinat["nivell_risc"] = df_combinat.apply(classifica_risc, axis=1)

# Mostrem les dades clau resultants
print(df_combinat[["state", "IA", "Rt", "theta", "pi", "positivity_rate", "pop_>65", "nivell_risc"]].head())


In [ ]:
df_combinat.head()

-----------------------------------------